In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.python import keras 
from keras import layers, models, optimizers
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16 
import os, shutil, random 
import pandas as pd
import pprint 
from collections import defaultdict

2025-07-15 01:25:09.096056: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-15 01:25:09.382248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-15 01:25:09.634189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752542709.823794   26031 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752542709.865843   26031 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752542710.397536   26031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [6]:
import errno 


base_directory= "/workspaces/machinelearning/animaldata"
output_directory= "/workspaces/machinelearning/output"
os.mkdir(output_directory)

train_directory= os.path.join(output_directory, "train")
try:
    os.mkdir(train_directory)
except OSError as exc: 
    if exc.errno !=errno.EEXIST:
        raise
    pass
test_directory= os.path.join(output_directory, "test")
os.mkdir(test_directory)

validation_directory= os.path.join(output_directory, "validation")
os.mkdir(validation_directory)

train_cats_directory = os.path.join(train_directory, 'cats')
os.mkdir(train_cats_directory)
train_dogs_directory = os.path.join(train_directory, 'dogs')
os.mkdir(train_dogs_directory)

test_cats_directory = os.path.join(test_directory, 'cats')
os.mkdir(test_cats_directory)
test_dogs_directory = os.path.join(test_directory, 'dogs')
os.mkdir(test_dogs_directory)

validation_cats_directory = os.path.join(validation_directory, 'cats')
os.mkdir(validation_cats_directory)
validation_dogs_directory = os.path.join(validation_directory, 'dogs')
os.mkdir(validation_dogs_directory)


cat_directory= "/workspaces/machinelearning/animaldata/Cat"
dog_directory= "/workspaces/machinelearning/animaldata/Dog"

fnamestrain= ["{}.jpg". format(i) for i in range(8750)]
fnamestest= ["{}.jpg". format(i) for i in range(8750, 11250)]
fnamesval= ["{}.jpg". format(i) for i in range(11250, 12499)]

def Calc(src, dst): 
    if not os.path.exists(src):
        return 
    if os.path.getsize(src)==0:
        return 

    try:
        shutil.copyfile(src, dst)
    except Exception as e:
        print(f"Broken file", src)

for fname in fnamestrain:
    Calc(os.path.join(cat_directory, fname), os.path.join(train_cats_directory, fname))
for fname in fnamestest:
    Calc(os.path.join(cat_directory, fname), os.path.join(test_cats_directory, fname))

for fname in fnamesval:
    Calc(os.path.join(cat_directory, fname), os.path.join(validation_cats_directory, fname))


for fname in fnamestrain:
    Calc(os.path.join(dog_directory, fname), os.path.join(train_dogs_directory, fname))
for fname in fnamestest:
    Calc(os.path.join(dog_directory, fname), os.path.join(test_dogs_directory, fname))

for fname in fnamesval:
    Calc(os.path.join(dog_directory, fname), os.path.join(validation_dogs_directory, fname))


In [7]:
def conv_block(filters):
    return [ 
        layers.Conv2D(filters, (3,3), activation= "relu", padding="same" ), 
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2)
    ]
model= models.Sequential()
model.add(layers.Input(shape=(150, 150, 3)))

for f in [32, 64, 128, 128]:
    for layer in conv_block(f):
        model.add(layer)

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", 
              optimizer= "adam",
              metrics= ["acc"])

2025-07-15 01:30:28.362972: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 150, 150, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 75, 75, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 37, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 18, 18, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,551,681 (21.18 MB)

 Trainable params: 5,550,977 (21.18 MB)

 Non-trainable params: 704 (2.75 KB)